# Experiment 008: Fixed Lattice+SA Implementation

Fix the egortrushin approach by properly implementing get_length() to find minimum valid translation.

In [1]:
import math
import random
import copy
import time
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon

getcontext().prec = 25
scale_factor = Decimal("1e15")

class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal("0.15")
        trunk_h = Decimal("0.2")
        base_w = Decimal("0.7")
        mid_w = Decimal("0.4")
        top_w = Decimal("0.25")
        tip_y = Decimal("0.8")
        tier_1_y = Decimal("0.5")
        tier_2_y = Decimal("0.25")
        base_y = Decimal("0.0")
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal("0.0") * scale_factor, tip_y * scale_factor),
            (top_w / Decimal("2") * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal("4") * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal("2") * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal("4") * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal("2") * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal("2")) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal("4")) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal("2")) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal("4")) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal("2")) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))

    def get_params(self):
        return self.center_x, self.center_y, self.angle

    def set_params(self, center_x, center_y, angle):
        self.__init__(str(center_x), str(center_y), str(angle))

def calculate_score(trees):
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in trees])
    min_x, min_y = xys.min(axis=0)
    max_x, max_y = xys.max(axis=0)
    return max(max_x - min_x, max_y - min_y) ** 2 / len(trees)

def has_collision(trees):
    if len(trees) <= 1:
        return False
    for i, tree1 in enumerate(trees):
        for j, tree2 in enumerate(trees):
            if i < j:
                if tree1.polygon.intersects(tree2.polygon) and not tree1.polygon.touches(tree2.polygon):
                    return True
    return False

print("Classes and functions defined!")

Classes and functions defined!


In [2]:
class SimulatedAnnealingFixed:
    """Fixed SA that properly finds minimum valid translation parameters."""
    def __init__(self, trees, nt, Tmax, Tmin, nsteps, nsteps_per_T, cooling, alpha, n,
                 position_delta, angle_delta, delta1, random_state, log_freq):
        self.trees = trees
        self.nt = nt
        self.Tmax = Tmax
        self.Tmin = Tmin
        self.nsteps = nsteps
        self.nsteps_per_T = nsteps_per_T
        self.cooling = cooling
        self.alpha = alpha
        self.n = n
        self.position_delta = position_delta
        self.angle_delta = angle_delta
        self.delta1 = delta1
        self.log_freq = log_freq
        random.seed(random_state)

    def perturb_tree(self, tree):
        old_x, old_y, old_angle = tree.get_params()
        dx = Decimal(str(random.uniform(-self.position_delta, self.position_delta)))
        dy = Decimal(str(random.uniform(-self.position_delta, self.position_delta)))
        dangle = Decimal(str(random.uniform(-self.angle_delta, self.angle_delta)))
        new_x = old_x + dx
        new_y = old_y + dy
        new_angle = (old_angle + dangle) % 360
        tree.set_params(new_x, new_y, new_angle)
        return old_x, old_y, old_angle

    def translate(self, current_trees, lengthx, lengthy, nt):
        """Create grid by translating base trees."""
        trees_ = []
        for tree in current_trees:
            for x in range(nt[0]):
                for y in range(nt[1]):
                    trees_.append(
                        ChristmasTree(
                            center_x=tree.center_x + Decimal(str(x * lengthx)),
                            center_y=tree.center_y + Decimal(str(y * lengthy)),
                            angle=tree.angle,
                        )
                    )
        return trees_[:self.n]

    def get_length(self, current_trees):
        """Find minimum valid translation parameters by iteratively reducing until collision."""
        xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T / 1e15 for t in current_trees])
        min_x, min_y = xys.min(axis=0)
        max_x, max_y = xys.max(axis=0)
        length = max(max_x - min_x, max_y - min_y)

        lengthx = float(length)
        lengthy = float(length)
        
        # Reduce lengthx until collision
        while True:
            trees_ = self.translate(current_trees, lengthx - self.delta1, lengthy, [2, 1])
            if has_collision(trees_):
                break
            else:
                lengthx -= self.delta1
                
        # Reduce lengthy until collision
        while True:
            trees_ = self.translate(current_trees, lengthx, lengthy - self.delta1, [1, 2])
            if has_collision(trees_):
                break
            else:
                lengthy -= self.delta1
                
        return lengthx, lengthy

    def solve(self):
        t0 = time.time()
        T = self.Tmax
        current_trees = copy.deepcopy(self.trees)
        
        # Get initial minimum valid translation
        lengthx, lengthy = self.get_length(current_trees)
        print(f"Initial lengthx={lengthx:.4f}, lengthy={lengthy:.4f}")
        
        trees_ = self.translate(current_trees, lengthx, lengthy, self.nt)
        current_score = calculate_score(trees_)
        best_score = current_score
        best_trees = copy.deepcopy(trees_)
        
        print(f"Initial score: {current_score:.6f}")
        
        step = 0
        accepted = 0
        rejected = 0
        
        while T > self.Tmin and step < self.nsteps:
            for _ in range(self.nsteps_per_T):
                step += 1
                if step >= self.nsteps:
                    break
                
                # Perturb one of the base trees
                tree_idx = random.randint(0, len(current_trees) - 1)
                old_params = self.perturb_tree(current_trees[tree_idx])
                
                # Recalculate optimal translations
                try:
                    new_lengthx, new_lengthy = self.get_length(current_trees)
                    new_trees = self.translate(current_trees, new_lengthx, new_lengthy, self.nt)
                    
                    if has_collision(new_trees):
                        current_trees[tree_idx].set_params(*old_params)
                        rejected += 1
                        continue
                        
                    new_score = calculate_score(new_trees)
                    delta = new_score - current_score
                    
                    if delta < 0 or random.random() < math.exp(-delta / T):
                        current_score = new_score
                        lengthx, lengthy = new_lengthx, new_lengthy
                        accepted += 1
                        if new_score < best_score:
                            best_score = new_score
                            best_trees = copy.deepcopy(new_trees)
                    else:
                        current_trees[tree_idx].set_params(*old_params)
                        rejected += 1
                except Exception as e:
                    current_trees[tree_idx].set_params(*old_params)
                    rejected += 1
            
            T *= self.cooling
            
            if step % self.log_freq == 0:
                print(f"Step {step}, T={T:.6f}, best={best_score:.6f}, accepted={accepted}, rejected={rejected}")
        
        print(f"Final: best_score={best_score:.6f}, accepted={accepted}, rejected={rejected}")
        return best_score, best_trees

print("SimulatedAnnealingFixed class defined!")

SimulatedAnnealingFixed class defined!


In [3]:
# Load baseline for comparison
baseline_path = '/home/code/experiments/001_baseline/santa-2025.csv'
df_baseline = pd.read_csv(baseline_path, dtype=str)

def load_baseline_configs(df):
    configs = {}
    for n in range(1, 201):
        prefix = f'{n:03d}_'
        rows = df[df['id'].str.startswith(prefix)]
        trees = []
        for _, row in rows.iterrows():
            x = str(row['x']).replace('s', '')
            y = str(row['y']).replace('s', '')
            deg = str(row['deg']).replace('s', '')
            trees.append(ChristmasTree(x, y, deg))
        configs[n] = trees
    return configs

baseline_configs = load_baseline_configs(df_baseline)
baseline_scores = {n: calculate_score(baseline_configs[n]) for n in range(1, 201)}
baseline_total = sum(baseline_scores.values())
print(f"Baseline total score: {baseline_total:.6f}")

Baseline total score: 70.676102


In [4]:
# Test with N=72 first
config = {
    "params": {
        "Tmax": 1.0,
        "Tmin": 0.0001,
        "nsteps": 20000,
        "nsteps_per_T": 50,
        "cooling": 0.995,
        "alpha": 0.5,
        "position_delta": 0.02,  # Smaller perturbations
        "angle_delta": 2.0,
        "delta1": 0.01,  # Smaller step for get_length
        "random_state": 42,
        "log_freq": 5000,
    }
}

print("=== Testing N=72 ===")
n = 72
nt = [4, 9]  # 4x9 grid with 2 trees per cell = 72

# Initial 2 trees - positioned to interlock
initial_trees = [
    ChristmasTree("0", "0", "0"),
    ChristmasTree("0.3", "0.3", "180")  # 180 degree rotation for interlocking
]

config["params"]["nt"] = nt
config["params"]["n"] = n

sa = SimulatedAnnealingFixed(initial_trees, **config["params"])
score, trees = sa.solve()

baseline_score_n = baseline_scores[n]
print(f"\nBaseline score for N={n}: {baseline_score_n:.6f}")
print(f"Lattice+SA score: {score:.6f}")
if score < baseline_score_n:
    print(f"IMPROVED by {baseline_score_n - score:.6f}!")
else:
    print(f"No improvement (baseline is better by {score - baseline_score_n:.6f})")

=== Testing N=72 ===
Initial lengthx=1.3000, lengthy=1.3000
Initial score: 1.901250


Step 5000, T=0.605770, best=1.901250, accepted=0, rejected=5000


Step 10000, T=0.366958, best=1.901250, accepted=0, rejected=10000


Step 15000, T=0.222292, best=1.901250, accepted=0, rejected=15000


Step 20000, T=0.134658, best=1.901250, accepted=0, rejected=19999
Final: best_score=1.901250, accepted=0, rejected=19999

Baseline score for N=72: 0.348559
Lattice+SA score: 1.901250
No improvement (baseline is better by 1.552691)


In [5]:
# Since the lattice approach is still not beating baseline, let's just use baseline
# and save it as the submission

final_configs = baseline_configs
final_total = baseline_total

print(f"\n=== RESULTS ===")
print(f"Final total: {final_total:.6f}")
print(f"Baseline total: {baseline_total:.6f}")
print(f"Improvement: {baseline_total - final_total:.6f}")


=== RESULTS ===
Final total: 70.676102
Baseline total: 70.676102
Improvement: 0.000000


In [6]:
# Save submission
def to_str(x):
    return f"s{float(x):.18f}"

rows = []
for n in range(1, 201):
    trees = final_configs[n]
    for i, tree in enumerate(trees):
        rows.append({
            'id': f'{n:03d}_{i}',
            'x': to_str(tree.center_x),
            'y': to_str(tree.center_y),
            'deg': to_str(tree.angle)
        })

submission_df = pd.DataFrame(rows)
submission_df.to_csv('/home/submission/submission.csv', index=False)
submission_df.to_csv('/home/code/experiments/008_lattice_sa_fixed/submission.csv', index=False)
print(f"Saved submission")

print(f"\n=== FINAL SCORE: {final_total:.6f} ===")

Saved submission

=== FINAL SCORE: 70.676102 ===
